In [ ]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.0 MB/s eta 0:00:00


In [ ]:
import cv2
import torch
import urllib.request
import os
import numpy as np
from google.colab import drive
import random

import matplotlib.pyplot as plt

In [ ]:
!pip install opencv-python

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


save each frame as images

In [ ]:
# Variables to save video details
num_frames = 0
video_height = 0
video_width = 0

# Video path
video_path = '/content/drive/My Drive/videos/StartToEnd.mp4'

# Ensure the folder exists
output_folder = '/content/drive/My Drive/video_images'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Open the video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Couldn't open the video file.")
else:
    # Get the width, height, and total frame count
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video width: {video_width}, height: {video_height}, total frames: {num_frames}")

    frame_number = 1
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save the frame as a .jpg file
        frame_file = os.path.join(output_folder, f"{frame_number}.jpg")
        cv2.imwrite(frame_file, frame)
        frame_number += 1

    cap.release()

print("Frames extraction completed!")


Video width: 1920, height: 1080, total frames: 41
Frames extraction completed!


Load a model (see [https://github.com/intel-isl/MiDaS/#Accuracy](https://github.com/intel-isl/MiDaS/#Accuracy) for an overview)

In [ ]:
# Inference size = 512
model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)


midas = torch.hub.load("intel-isl/MiDaS", model_type)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v3/dpt_large_384.pt" to /root/.cache/torch/hub/checkpoints/dpt_large_384.pt
100%|██████████| 1.28G/1.28G [00:05<00:00, 268MB/s]


Move model to GPU if available

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

DPTDepthModel(
  (pretrained): Module(
    (model): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_featur

another operation here

In [ ]:
# Load the MiDaS model and transforms
midas = torch.hub.load("intel-isl/MiDaS", model_type)
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

# Set directories
input_folder = '/content/drive/My Drive/video_images'
output_folder = '/content/drive/My Drive/disparity_map_outputs'

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the input folder
for image_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, image_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    input_batch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    output = prediction.cpu().numpy()

    # Normalize the disparity map for visualization and save
    normalized_output = ((output - output.min()) / (output.max() - output.min()) * 255).astype(np.uint8)
    output_img_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_img_path, normalized_output)

print("Disparity map creation completed!")

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


Disparity map creation completed!


checking for height and width

In [ ]:
# Directory
output_folder = '/content/drive/My Drive/disparity_map_outputs'

# Check each image in the directory
for image_name in os.listdir(output_folder):
    img_path = os.path.join(output_folder, image_name)
    img = cv2.imread(img_path)

    if img is not None:
        height, width = img.shape[:2]
        print(f"Image {image_name}: Width = {width}, Height = {height}")
    else:
        print(f"Unable to read {image_name}.")

Image 3.jpg: Width = 1920, Height = 1080
Image 1.jpg: Width = 1920, Height = 1080
Image 2.jpg: Width = 1920, Height = 1080
Image 6.jpg: Width = 1920, Height = 1080
Image 4.jpg: Width = 1920, Height = 1080
Image 9.jpg: Width = 1920, Height = 1080
Image 8.jpg: Width = 1920, Height = 1080
Image 7.jpg: Width = 1920, Height = 1080
Image 5.jpg: Width = 1920, Height = 1080
Image 10.jpg: Width = 1920, Height = 1080
Image 11.jpg: Width = 1920, Height = 1080
Image 12.jpg: Width = 1920, Height = 1080
Image 13.jpg: Width = 1920, Height = 1080
Image 15.jpg: Width = 1920, Height = 1080
Image 14.jpg: Width = 1920, Height = 1080
Image 16.jpg: Width = 1920, Height = 1080
Image 17.jpg: Width = 1920, Height = 1080
Image 18.jpg: Width = 1920, Height = 1080
Image 19.jpg: Width = 1920, Height = 1080
Image 20.jpg: Width = 1920, Height = 1080
Image 21.jpg: Width = 1920, Height = 1080
Image 22.jpg: Width = 1920, Height = 1080
Image 24.jpg: Width = 1920, Height = 1080
Image 23.jpg: Width = 1920, Height = 1080
I

Extract frame rate from the original video:

In [ ]:
# Video path
original_video_path = '/content/drive/My Drive/videos/StartToEnd.mp4'

# Extract frame rate
cap = cv2.VideoCapture(original_video_path)
if not cap.isOpened():
    print("Error: Couldn't open the original video file.")
    fps = 30  # Default to 30 if there's an error.
else:
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    cap.release()

Generate the new video using the extracted frame rate:

In [ ]:
output_folder = '/content/drive/My Drive/disparity_map_outputs'
video_path = '/content/drive/My Drive/disparity_map_video_.mp4'

# Get list of image files
image_files = [os.path.join(output_folder, fname) for fname in os.listdir(output_folder) if fname.endswith('.jpg')]

# Sort the files based on their integer values
image_files = sorted(image_files, key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))

# Check the first image to get video dimensions
first_image = cv2.imread(image_files[0])
height, width, layers = first_image.shape

# Initialize the video writer with the frame rate from the original video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

# Read each file and write to video
for image_file in image_files:
    img = cv2.imread(image_file)
    out.write(img)

out.release()

print(f"Video saved to {video_path} with frame rate: {fps}")

Video saved to /content/drive/My Drive/disparity_map_video_.mp4 with frame rate: 30


In [ ]:
# Pre-allocate the array
depth_maps = np.zeros((num_frames, video_height, video_width), dtype=np.uint8)

# Process each image in the input folder
for idx, image_name in enumerate(sorted(os.listdir(input_folder), key=lambda x: int(os.path.splitext(x)[0]))):  # Sorting the images
    img_path = os.path.join(input_folder, image_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    input_batch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    output = prediction.cpu().numpy()

    # Print original output stats
    print(f"Image: {image_name}")
    print("Original Output:")
    print(f"Min: {np.min(output)}, Max: {np.max(output)}, Avg: {np.mean(output)}")

    # Normalize the disparity map for visualization
    normalized_output = ((output - output.min()) / (output.max() - output.min()) * 255).astype(np.uint8)

    # Print normalized output stats
    print("Normalized Output:")
    print(f"Min: {np.min(normalized_output)}, Max: {np.max(normalized_output)}, Avg: {np.mean(normalized_output)}")
    print("-" * 50)

    # Insert the normalized output into the pre-allocated array
    depth_maps[idx, :, :] = normalized_output

    output_img_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_img_path, normalized_output)


Image: 1.jpg
Original Output:
Min: -1.8285777568817139, Max: 40.2906379699707, Avg: 12.149803161621094
Normalized Output:
Min: 0, Max: 255, Avg: 84.13270351080247
--------------------------------------------------
Image: 2.jpg
Original Output:
Min: -1.6654014587402344, Max: 40.24309539794922, Avg: 12.249217987060547
Normalized Output:
Min: 0, Max: 255, Avg: 84.16885561342593
--------------------------------------------------
Image: 3.jpg
Original Output:
Min: -1.9754635095596313, Max: 40.232032775878906, Avg: 12.370478630065918
Normalized Output:
Min: 0, Max: 255, Avg: 86.1696981095679
--------------------------------------------------
Image: 4.jpg
Original Output:
Min: -1.7526516914367676, Max: 39.84197998046875, Avg: 12.300341606140137
Normalized Output:
Min: 0, Max: 255, Avg: 85.65256269290123
--------------------------------------------------
Image: 5.jpg
Original Output:
Min: -1.7449167966842651, Max: 40.085601806640625, Avg: 12.325409889221191
Normalized Output:
Min: 0, Max: 255,

In [ ]:
"""
# Load the image
image_path = os.path.join(input_folder, "6.jpg")
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Extract corresponding depth map values
depth_map_for_image = depth_maps[5]  # 0-based indexing, so index 5 corresponds to 6.jpg

# Randomly select 20 points
num_points = 20
random_points = [(random.randint(0, video_height-1), random.randint(0, video_width-1)) for _ in range(num_points)]

plt.figure(figsize=(10, 10))

# Display the image
plt.imshow(image)

# Overlay points and depth values
for y, x in random_points:
    depth_value = depth_map_for_image[y, x]
    plt.scatter(x, y, color='red', s=50)  # Plotting the point
    plt.text(x, y, str(depth_value), color='white', fontsize=8, ha='center', va='center')  # Annotating the point with its depth value

plt.axis('off')
plt.show()
"""

'\n# Load the image\nimage_path = os.path.join(input_folder, "6.jpg")\nimage = cv2.imread(image_path)\nimage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n\n# Extract corresponding depth map values\ndepth_map_for_image = depth_maps[5]  # 0-based indexing, so index 5 corresponds to 6.jpg\n\n# Randomly select 20 points\nnum_points = 20\nrandom_points = [(random.randint(0, video_height-1), random.randint(0, video_width-1)) for _ in range(num_points)]\n\nplt.figure(figsize=(10, 10))\n\n# Display the image\nplt.imshow(image)\n\n# Overlay points and depth values\nfor y, x in random_points:\n    depth_value = depth_map_for_image[y, x]\n    plt.scatter(x, y, color=\'red\', s=50)  # Plotting the point\n    plt.text(x, y, str(depth_value), color=\'white\', fontsize=8, ha=\'center\', va=\'center\')  # Annotating the point with its depth value\n\nplt.axis(\'off\')\nplt.show()\n'

In [ ]:
"""
def plot_image_with_depth_points(image, depth_map):
    # Randomly select 20 points
    num_points = 20
    random_points = [(random.randint(0, video_height-1), random.randint(0, video_width-1)) for _ in range(num_points)]

    plt.figure(figsize=(10, 10))

    # Display the image
    plt.imshow(image)

    # Overlay points and depth values
    for y, x in random_points:
        depth_value = depth_map[y, x]
        plt.scatter(x, y, color='red', s=50)  # Plotting the point
        plt.text(x, y, str(depth_value), color='white', fontsize=8, ha='center', va='center')  # Annotating the point with its depth value

    plt.axis('off')
    plt.show()

# Process each image in the input folder
for idx, image_name in enumerate(sorted(os.listdir(input_folder), key=lambda x: int(os.path.splitext(x)[0]))):  # Sorting the images based on their number
    # Load the image
    image_path = os.path.join(input_folder, image_name)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Extract corresponding depth map values
    depth_map_for_image = depth_maps[idx]

    # Display image with depth points
    plot_image_with_depth_points(image, depth_map_for_image)
  """


"\ndef plot_image_with_depth_points(image, depth_map):\n    # Randomly select 20 points\n    num_points = 20\n    random_points = [(random.randint(0, video_height-1), random.randint(0, video_width-1)) for _ in range(num_points)]\n\n    plt.figure(figsize=(10, 10))\n\n    # Display the image\n    plt.imshow(image)\n\n    # Overlay points and depth values\n    for y, x in random_points:\n        depth_value = depth_map[y, x]\n        plt.scatter(x, y, color='red', s=50)  # Plotting the point\n        plt.text(x, y, str(depth_value), color='white', fontsize=8, ha='center', va='center')  # Annotating the point with its depth value\n\n    plt.axis('off')\n    plt.show()\n\n# Process each image in the input folder\nfor idx, image_name in enumerate(sorted(os.listdir(input_folder), key=lambda x: int(os.path.splitext(x)[0]))):  # Sorting the images based on their number\n    # Load the image\n    image_path = os.path.join(input_folder, image_name)\n    image = cv2.imread(image_path)\n    image 

In [ ]:
print(depth_maps)

[[[ 21  21  21 ...  25  25  25]
  [ 21  21  21 ...  25  25  25]
  [ 21  21  21 ...  25  25  25]
  ...
  [191 191 191 ... 254 254 254]
  [191 191 191 ... 254 254 254]
  [191 191 191 ... 254 254 254]]

 [[ 20  20  20 ...  24  24  24]
  [ 20  20  20 ...  24  24  24]
  [ 20  20  20 ...  23  23  23]
  ...
  [193 193 193 ... 253 253 253]
  [193 193 193 ... 254 254 254]
  [193 193 193 ... 254 254 254]]

 [[ 22  22  22 ...  26  26  26]
  [ 22  22  22 ...  26  26  26]
  [ 22  22  22 ...  26  26  26]
  ...
  [199 199 199 ... 254 254 254]
  [199 199 199 ... 254 254 254]
  [199 199 199 ... 254 254 254]]

 ...

 [[ 30  30  30 ...  38  38  38]
  [ 30  30  30 ...  38  38  38]
  [ 30  30  30 ...  38  38  38]
  ...
  [208 208 208 ... 254 254 254]
  [209 209 209 ... 254 254 254]
  [209 209 209 ... 254 254 254]]

 [[ 28  28  28 ...  33  33  33]
  [ 28  28  28 ...  33  33  33]
  [ 28  28  28 ...  33  32  32]
  ...
  [206 206 206 ... 254 254 254]
  [207 207 207 ... 254 254 254]
  [207 207 207 ... 254 254 2

In [ ]:
"""
print(len(depth_maps[0]))
print(len(depth_maps[1]))
print(len(depth_maps[2]))
print(len(depth_maps[3]))
print(len(depth_maps[4]))
"""

'\nprint(len(depth_maps[0]))\nprint(len(depth_maps[1]))\nprint(len(depth_maps[2]))\nprint(len(depth_maps[3]))\nprint(len(depth_maps[4]))\n'

In [ ]:
frames, height, width = depth_maps.shape

print(f"Number of frames: {frames}")
print(f"Height of the video: {height} pixels")
print(f"Width of each frame: {width} pixels")

Number of frames: 41
Height of the video: 1080 pixels
Width of each frame: 1920 pixels


In [ ]:
"""
def plot_image_with_depth_point(image, depth_map, x, y):
    plt.figure(figsize=(10, 10))

    # Display the image
    plt.imshow(image)

    # Make the specified pixel white
    depth_value = depth_map[y, x]
    plt.scatter(x, y, color='white', s=50)  # Plotting the point as white
    plt.text(x, y, str(depth_value), color='black', fontsize=8, ha='center', va='center')  # Annotating the point with its depth value

    plt.axis('off')
    plt.show()


for idx, image_name in enumerate(sorted(os.listdir(input_folder), key=lambda x: int(os.path.splitext(x)[0]))):
    # Load the image
    image_path = os.path.join(input_folder, image_name)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Extract corresponding depth map values
    depth_map_for_image = depth_maps[idx]

    # Call your modified function to display the image with depth points
    # Example x and y coordinates (replace with your desired values)
    x = width - 1820
    y = height - 10
    plot_image_with_depth_point(image, depth_map_for_image, x, y)
"""

"\ndef plot_image_with_depth_point(image, depth_map, x, y):\n    plt.figure(figsize=(10, 10))\n\n    # Display the image\n    plt.imshow(image)\n\n    # Make the specified pixel white\n    depth_value = depth_map[y, x]\n    plt.scatter(x, y, color='white', s=50)  # Plotting the point as white\n    plt.text(x, y, str(depth_value), color='black', fontsize=8, ha='center', va='center')  # Annotating the point with its depth value\n\n    plt.axis('off')\n    plt.show()\n\n\nfor idx, image_name in enumerate(sorted(os.listdir(input_folder), key=lambda x: int(os.path.splitext(x)[0]))):\n    # Load the image\n    image_path = os.path.join(input_folder, image_name)\n    image = cv2.imread(image_path)\n    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n\n    # Extract corresponding depth map values\n    depth_map_for_image = depth_maps[idx]\n\n    # Call your modified function to display the image with depth points\n    # Example x and y coordinates (replace with your desired values)\n    x = wi

In [ ]:
max_value = np.max(depth_maps)
min_value = np.min(depth_maps)

print(f"Maximum value in the matrix: {max_value}")
print(f"Minimum value in the matrix: {min_value}")

Maximum value in the matrix: 255
Minimum value in the matrix: 0


In [ ]:
modified_depth_maps = 255 - depth_maps
print(modified_depth_maps)

[[[234 234 234 ... 230 230 230]
  [234 234 234 ... 230 230 230]
  [234 234 234 ... 230 230 230]
  ...
  [ 64  64  64 ...   1   1   1]
  [ 64  64  64 ...   1   1   1]
  [ 64  64  64 ...   1   1   1]]

 [[235 235 235 ... 231 231 231]
  [235 235 235 ... 231 231 231]
  [235 235 235 ... 232 232 232]
  ...
  [ 62  62  62 ...   2   2   2]
  [ 62  62  62 ...   1   1   1]
  [ 62  62  62 ...   1   1   1]]

 [[233 233 233 ... 229 229 229]
  [233 233 233 ... 229 229 229]
  [233 233 233 ... 229 229 229]
  ...
  [ 56  56  56 ...   1   1   1]
  [ 56  56  56 ...   1   1   1]
  [ 56  56  56 ...   1   1   1]]

 ...

 [[225 225 225 ... 217 217 217]
  [225 225 225 ... 217 217 217]
  [225 225 225 ... 217 217 217]
  ...
  [ 47  47  47 ...   1   1   1]
  [ 46  46  46 ...   1   1   1]
  [ 46  46  46 ...   1   1   1]]

 [[227 227 227 ... 222 222 222]
  [227 227 227 ... 222 222 222]
  [227 227 227 ... 222 223 223]
  ...
  [ 49  49  49 ...   1   1   1]
  [ 48  48  48 ...   1   1   1]
  [ 48  48  48 ...   1   1  

In [ ]:
"""
for idx, image_name in enumerate(sorted(os.listdir(input_folder), key=lambda x: int(os.path.splitext(x)[0]))):
    # Load the image
    image_path = os.path.join(input_folder, image_name)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    depth_map_for_image = modified_depth_maps[idx]


    x = width - 1820
    y = height - 10
    plot_image_with_depth_point(image, depth_map_for_image, x, y)
"""

'\nfor idx, image_name in enumerate(sorted(os.listdir(input_folder), key=lambda x: int(os.path.splitext(x)[0]))):\n    # Load the image\n    image_path = os.path.join(input_folder, image_name)\n    image = cv2.imread(image_path)\n    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n\n    depth_map_for_image = modified_depth_maps[idx]\n\n\n    x = width - 1820\n    y = height - 10\n    plot_image_with_depth_point(image, depth_map_for_image, x, y)\n'

In [ ]:
final_disparity_depth_map = modified_depth_maps.astype(np.uint16)
print(final_disparity_depth_map)

max_value = np.max(final_disparity_depth_map)
min_value = np.min(final_disparity_depth_map)

print(f"Maximum value in the matrix: {max_value}")
print(f"Minimum value in the matrix: {min_value}")


frames, height, width = final_disparity_depth_map.shape

print(f"Number of frames: {frames}")
print(f"Height of the video: {height} pixels")
print(f"Width of each frame: {width} pixels")

[[[234 234 234 ... 230 230 230]
  [234 234 234 ... 230 230 230]
  [234 234 234 ... 230 230 230]
  ...
  [ 64  64  64 ...   1   1   1]
  [ 64  64  64 ...   1   1   1]
  [ 64  64  64 ...   1   1   1]]

 [[235 235 235 ... 231 231 231]
  [235 235 235 ... 231 231 231]
  [235 235 235 ... 232 232 232]
  ...
  [ 62  62  62 ...   2   2   2]
  [ 62  62  62 ...   1   1   1]
  [ 62  62  62 ...   1   1   1]]

 [[233 233 233 ... 229 229 229]
  [233 233 233 ... 229 229 229]
  [233 233 233 ... 229 229 229]
  ...
  [ 56  56  56 ...   1   1   1]
  [ 56  56  56 ...   1   1   1]
  [ 56  56  56 ...   1   1   1]]

 ...

 [[225 225 225 ... 217 217 217]
  [225 225 225 ... 217 217 217]
  [225 225 225 ... 217 217 217]
  ...
  [ 47  47  47 ...   1   1   1]
  [ 46  46  46 ...   1   1   1]
  [ 46  46  46 ...   1   1   1]]

 [[227 227 227 ... 222 222 222]
  [227 227 227 ... 222 222 222]
  [227 227 227 ... 222 223 223]
  ...
  [ 49  49  49 ...   1   1   1]
  [ 48  48  48 ...   1   1   1]
  [ 48  48  48 ...   1   1  

In [ ]:
real_distance = 200
multipliers = []

# Iterate over each inner matrix in final_disparity_depth_map
for inner_matrix in final_disparity_depth_map:
    # Assuming you want x and y coordinates to be (width - 1820) and (height - 10) for each inner matrix
    x = inner_matrix.shape[1] - 1820
    y = inner_matrix.shape[0] - 10

    # Calculate the multiplier for this inner matrix
    multiplier = real_distance / inner_matrix[y, x]

    # Add the multiplier to the list
    multipliers.append(multiplier)


In [ ]:
print(multipliers)

[3.1746031746031744, 3.278688524590164, 3.508771929824561, 3.508771929824561, 3.4482758620689653, 3.5714285714285716, 3.5714285714285716, 3.7037037037037037, 3.6363636363636362, 3.8461538461538463, 4.081632653061225, 4.0, 4.166666666666667, 4.081632653061225, 4.081632653061225, 3.9215686274509802, 3.6363636363636362, 3.8461538461538463, 3.9215686274509802, 3.7735849056603774, 3.7735849056603774, 3.7735849056603774, 3.8461538461538463, 3.7735849056603774, 3.7037037037037037, 3.8461538461538463, 4.0, 3.9215686274509802, 3.6363636363636362, 4.0, 4.081632653061225, 3.9215686274509802, 4.0, 4.081632653061225, 4.081632653061225, 4.081632653061225, 3.9215686274509802, 4.081632653061225, 3.9215686274509802, 3.8461538461538463, 3.6363636363636362]


In [ ]:
print(len(multipliers))

41


In [ ]:
final_disparity = final_disparity_depth_map.copy()

In [ ]:
print(final_disparity)

[[[234 234 234 ... 230 230 230]
  [234 234 234 ... 230 230 230]
  [234 234 234 ... 230 230 230]
  ...
  [ 64  64  64 ...   1   1   1]
  [ 64  64  64 ...   1   1   1]
  [ 64  64  64 ...   1   1   1]]

 [[235 235 235 ... 231 231 231]
  [235 235 235 ... 231 231 231]
  [235 235 235 ... 232 232 232]
  ...
  [ 62  62  62 ...   2   2   2]
  [ 62  62  62 ...   1   1   1]
  [ 62  62  62 ...   1   1   1]]

 [[233 233 233 ... 229 229 229]
  [233 233 233 ... 229 229 229]
  [233 233 233 ... 229 229 229]
  ...
  [ 56  56  56 ...   1   1   1]
  [ 56  56  56 ...   1   1   1]
  [ 56  56  56 ...   1   1   1]]

 ...

 [[225 225 225 ... 217 217 217]
  [225 225 225 ... 217 217 217]
  [225 225 225 ... 217 217 217]
  ...
  [ 47  47  47 ...   1   1   1]
  [ 46  46  46 ...   1   1   1]
  [ 46  46  46 ...   1   1   1]]

 [[227 227 227 ... 222 222 222]
  [227 227 227 ... 222 222 222]
  [227 227 227 ... 222 223 223]
  ...
  [ 49  49  49 ...   1   1   1]
  [ 48  48  48 ...   1   1   1]
  [ 48  48  48 ...   1   1  

In [ ]:
for i, inner_matrix in enumerate(final_disparity):
    final_disparity[i] = inner_matrix * multipliers[i]

In [ ]:
print(final_disparity)

[[[742 742 742 ... 730 730 730]
  [742 742 742 ... 730 730 730]
  [742 742 742 ... 730 730 730]
  ...
  [203 203 203 ...   3   3   3]
  [203 203 203 ...   3   3   3]
  [203 203 203 ...   3   3   3]]

 [[770 770 770 ... 757 757 757]
  [770 770 770 ... 757 757 757]
  [770 770 770 ... 760 760 760]
  ...
  [203 203 203 ...   6   6   6]
  [203 203 203 ...   3   3   3]
  [203 203 203 ...   3   3   3]]

 [[817 817 817 ... 803 803 803]
  [817 817 817 ... 803 803 803]
  [817 817 817 ... 803 803 803]
  ...
  [196 196 196 ...   3   3   3]
  [196 196 196 ...   3   3   3]
  [196 196 196 ...   3   3   3]]

 ...

 [[882 882 882 ... 850 850 850]
  [882 882 882 ... 850 850 850]
  [882 882 882 ... 850 850 850]
  ...
  [184 184 184 ...   3   3   3]
  [180 180 180 ...   3   3   3]
  [180 180 180 ...   3   3   3]]

 [[873 873 873 ... 853 853 853]
  [873 873 873 ... 853 853 853]
  [873 873 873 ... 853 857 857]
  ...
  [188 188 188 ...   3   3   3]
  [184 184 184 ...   3   3   3]
  [184 184 184 ...   3   3  

In [ ]:
# download final_disparity.py from folders
np.save('final_disparity.npy', final_disparity)

In [ ]:
print(final_disparity[0][0][0])
print(final_disparity[1][0][0])
print(final_disparity[2][0][0])
print(final_disparity[2][10][1081])

742
770
817
768


### References
[Towards Robust Monocular Depth Estimation: Mixing Datasets for Zero-shot Cross-dataset Transfer](https://arxiv.org/abs/1907.01341)

[Vision Transformers for Dense Prediction](https://arxiv.org/abs/2103.13413)

Please cite our papers if you use our models:

In [ ]:
@article{Ranftl2020,
	author    = {Ren\'{e} Ranftl and Katrin Lasinger and David Hafner and Konrad Schindler and Vladlen Koltun},
	title     = {Towards Robust Monocular Depth Estimation: Mixing Datasets for Zero-shot Cross-dataset Transfer},
	journal   = {IEEE Transactions on Pattern Analysis and Machine Intelligence (TPAMI)},
	year      = {2020},
}

In [ ]:
@article{Ranftl2021,
	author    = {Ren\'{e} Ranftl and Alexey Bochkovskiy and Vladlen Koltun},
	title     = {Vision Transformers for Dense Prediction},
	journal   = {ArXiv preprint},
	year      = {2021},
}